In [23]:
from gurobipy import *
import numpy as np
import pandas as pd
import sys

## Load model

In [24]:
model = read("./model/2.2.1.mps")
model.write("./model/2.2.1/model.lp")
print(f"Number of vars: {len(model.getVars())}")
print(f"Number of constraints: {len(model.getConstrs())}")
print(model.getVars())

Read MPS format model from file ./model/2.2.1.mps
Reading time = 0.01 seconds
2.2.1: 210 rows, 2010 columns, 4010 nonzeros
Number of vars: 2010
Number of constraints: 210
[<gurobi.Var x[0,0]>, <gurobi.Var x[0,1]>, <gurobi.Var x[0,2]>, <gurobi.Var x[0,3]>, <gurobi.Var x[0,4]>, <gurobi.Var x[0,5]>, <gurobi.Var x[0,6]>, <gurobi.Var x[0,7]>, <gurobi.Var x[0,8]>, <gurobi.Var x[0,9]>, <gurobi.Var x[0,10]>, <gurobi.Var x[0,11]>, <gurobi.Var x[0,12]>, <gurobi.Var x[0,13]>, <gurobi.Var x[0,14]>, <gurobi.Var x[0,15]>, <gurobi.Var x[0,16]>, <gurobi.Var x[0,17]>, <gurobi.Var x[0,18]>, <gurobi.Var x[0,19]>, <gurobi.Var x[0,20]>, <gurobi.Var x[0,21]>, <gurobi.Var x[0,22]>, <gurobi.Var x[0,23]>, <gurobi.Var x[0,24]>, <gurobi.Var x[0,25]>, <gurobi.Var x[0,26]>, <gurobi.Var x[0,27]>, <gurobi.Var x[0,28]>, <gurobi.Var x[0,29]>, <gurobi.Var x[0,30]>, <gurobi.Var x[0,31]>, <gurobi.Var x[0,32]>, <gurobi.Var x[0,33]>, <gurobi.Var x[0,34]>, <gurobi.Var x[0,35]>, <gurobi.Var x[0,36]>, <gurobi.Var x[0,37]>, <g

## Read file

In [25]:
f = pd.read_csv("./data/2.5/Facility.csv").iloc[:, 1:].to_numpy()
f

array([[5.21e+00, 8.07e+00, 1.00e+03, 6.80e+02],
       [4.40e+00, 7.31e+00, 1.00e+03, 7.00e+02],
       [3.26e+00, 6.01e+00, 1.00e+03, 6.80e+02],
       [3.02e+00, 5.26e+00, 1.00e+03, 6.10e+02],
       [1.07e+00, 3.86e+00, 1.00e+03, 5.40e+02],
       [1.52e+00, 2.50e+00, 1.00e+03, 5.00e+02],
       [4.92e+00, 1.60e-01, 1.00e+03, 5.90e+02],
       [5.07e+00, 1.34e+00, 1.00e+03, 5.80e+02],
       [7.85e+00, 4.30e+00, 1.00e+03, 6.00e+02],
       [8.12e+00, 4.37e+00, 1.00e+03, 6.00e+02]])

In [26]:
D = pd.read_csv("./data/2.5/DemandRand.csv").iloc[:, 1:].to_numpy()
D

array([[ 3.616,  4.262, 28.   ,  2.266],
       [ 2.29 ,  3.025, 28.   ,  2.408],
       [ 2.757,  2.403, 35.   ,  3.211],
       [ 2.882,  3.153, 34.   ,  3.208],
       [ 3.552,  3.725, 31.   ,  3.375],
       [ 3.623,  4.158, 23.   ,  2.283],
       [ 2.94 ,  4.729, 17.   ,  1.518],
       [ 2.377,  4.115, 44.   ,  3.185],
       [ 3.617,  3.784, 19.   ,  1.633],
       [ 2.736,  2.716, 15.   ,  1.536],
       [ 4.836,  4.996,  9.   ,  1.017],
       [ 3.287,  4.179, 44.   ,  3.237],
       [ 5.077,  4.778, 12.   ,  1.569],
       [ 2.56 ,  2.246, 19.   ,  1.631],
       [ 2.83 ,  4.729,  9.   ,  1.096],
       [ 2.603,  3.376, 15.   ,  1.725],
       [ 5.336,  3.708, 22.   ,  2.467],
       [ 4.49 ,  3.064, 41.   ,  3.19 ],
       [ 4.473,  4.6  , 19.   ,  1.572],
       [ 2.348,  4.35 , 44.   ,  3.285],
       [ 5.435,  4.345,  7.   ,  1.033],
       [ 3.182,  4.466, 20.   ,  1.508],
       [ 2.618,  3.456, 20.   ,  1.831],
       [ 5.179,  2.999, 35.   ,  3.302],
       [ 3.205, 

In [27]:
t = pd.read_csv("./data/2.5/TransCost.csv", header=None).to_numpy()
t

array([[4.13, 5.83, 6.18, ..., 2.21, 2.45, 3.52],
       [3.15, 4.78, 5.17, ..., 2.22, 2.1 , 3.13],
       [1.78, 3.14, 3.64, ..., 3.02, 2.47, 3.11],
       ...,
       [3.26, 3.25, 2.55, ..., 4.94, 4.38, 3.46],
       [4.23, 5.7 , 5.43, ..., 2.41, 2.54, 1.83],
       [4.51, 5.98, 5.71, ..., 2.55, 2.74, 2.08]])

## Read vars

In [28]:
x = {}
c = {}

for i in range(10):
    c[i] = model.getVarByName(f"chose[{i}]")
    for j in range(200):
        x[i, j] = model.getVarByName(f"x[{i},{j}]")

print(x)
print(c)

{(0, 0): <gurobi.Var x[0,0]>, (0, 1): <gurobi.Var x[0,1]>, (0, 2): <gurobi.Var x[0,2]>, (0, 3): <gurobi.Var x[0,3]>, (0, 4): <gurobi.Var x[0,4]>, (0, 5): <gurobi.Var x[0,5]>, (0, 6): <gurobi.Var x[0,6]>, (0, 7): <gurobi.Var x[0,7]>, (0, 8): <gurobi.Var x[0,8]>, (0, 9): <gurobi.Var x[0,9]>, (0, 10): <gurobi.Var x[0,10]>, (0, 11): <gurobi.Var x[0,11]>, (0, 12): <gurobi.Var x[0,12]>, (0, 13): <gurobi.Var x[0,13]>, (0, 14): <gurobi.Var x[0,14]>, (0, 15): <gurobi.Var x[0,15]>, (0, 16): <gurobi.Var x[0,16]>, (0, 17): <gurobi.Var x[0,17]>, (0, 18): <gurobi.Var x[0,18]>, (0, 19): <gurobi.Var x[0,19]>, (0, 20): <gurobi.Var x[0,20]>, (0, 21): <gurobi.Var x[0,21]>, (0, 22): <gurobi.Var x[0,22]>, (0, 23): <gurobi.Var x[0,23]>, (0, 24): <gurobi.Var x[0,24]>, (0, 25): <gurobi.Var x[0,25]>, (0, 26): <gurobi.Var x[0,26]>, (0, 27): <gurobi.Var x[0,27]>, (0, 28): <gurobi.Var x[0,28]>, (0, 29): <gurobi.Var x[0,29]>, (0, 30): <gurobi.Var x[0,30]>, (0, 31): <gurobi.Var x[0,31]>, (0, 32): <gurobi.Var x[0,32

## Initialize values for model

In [29]:
model.optimize() # Run the model to get the initial values

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Optimize a model with 210 rows, 2010 columns and 4010 nonzeros
Model fingerprint: 0xd7de5ed0
Variable types: 0 continuous, 2010 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e-01, 7e+02]
  Bounds range     [1e+00, 1e+03]
  RHS range        [7e+00, 4e+01]
Found heuristic solution: objective 23918.380000
Presolve time: 0.01s
Presolved: 210 rows, 2010 columns, 4010 nonzeros
Variable types: 0 continuous, 2010 integer (10 binary)
Found heuristic solution: objective 23657.160000

Root relaxation: objective 1.093869e+04, 239 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objecti

## Calculate the demand

In [30]:
D = np.concatenate((D, np.array([np.round(np.random.normal(D[:, 2], D[:, 3], len(D[:, 2])))]).T), axis=1)
with open("./data/2.5/Demand.csv", "w") as f:
    for i in D[:, -1]:
        f.write(str(i) + "\n")

## Update the demand

In [31]:
for j in range(200):
    temp = model.getConstrByName(f"shop-demand[{j}]")
    if temp: model.remove(temp)
    model.addConstr(quicksum(x[i, j] for i in range(10)) >= D[j, -1])

model.write("./model/2.5/model.lp")

## Run again

In [32]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Optimize a model with 410 rows, 2010 columns and 6010 nonzeros
Model fingerprint: 0xfc753b7b
Variable types: 0 continuous, 2010 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e-01, 7e+02]
  Bounds range     [1e+00, 1e+03]
  RHS range        [5e+00, 5e+01]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R210 by 5.000000000

Found heuristic solution: objective 24497.720000
Presolve removed 200 rows and 0 columns
Presolve time: 0.01s
Presolved: 210 rows, 2010 columns, 4010 nonzeros
Variable types: 0 continuous, 2010 integer (10 binary)
Fou

In [33]:
if model.status == GRB.Status.OPTIMAL:
    temp = sys.stdout
    sys.stdout = open("./out/2.5.out", "w")
    print(0, end=" ")
    for i in range(10):
        print(round(c[i].X), end=" ")
    print()
    for i in range(10):
        if c[i].X:
            for j in range(200):
                if x[i, j].X != 0:
                    print(f"{i + 1} {j + 1} {x[i, j].X}")
    print(model.ObjVal)
    sys.stdout = temp